# 调度专业知识问答系统
## 集训前期提供过调度知识和人工智能的相关题库，每一个问题在excel中可找到相应的答案，即每一个样本数据是 <问题，答案>。 系统的核心思想是根据文本相似度，当用户输入一个问题，从题库中找到最相似的若干题目，然后直接返回相应的答案即可。
### 考题1-6
#### 1.定义删除母,数字，汉字以外的所有符号的函数remove_punctuation（），补充相关代码。（10分）
#### 2.读取文本数据后，采用正则化方法，调用stopwords（）和remove_punctuation（），补充相关代码。（10分）
#### 3.建立R行C列的矩阵TfIdf和OneHot，对应存放tfidf向量表示和01向量表示，R为文档样本数，这里是行数，C为不重复词语数，即编码维度，补充相关代码。（30分）
#### 4.给定用户问题的单词列表query，通过向量表示Vector和词表words_vocab生成用户问题的向量表示vector_query，补充相关代码。（10分）
#### 5.采用余弦相似度、欧式距离和曼哈顿距离计算句子间相似度，补充相关代码。（15分）
#### 6.#使用函数cosine_similarity求出与用户问题最相似的问题，排序输出和用户问题相似度最高的前3个问题的答案，补充相关代码。（25分）
    注意：这里我们采用TfIdf的向量表示，


## Part1：文本预处理

In [28]:
import pandas as pd
import numpy as np
import jieba
import re
import math

In [29]:
# 采用正则化方法，定义删除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    #######***************************答题区1***************************#######
    ###-----------------------------------10分
    if line.strip() == '':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('', line)
    #######************************************************************#######
    return line

In [30]:
#答题1测试
line_test="《电力监控系统---安全防护123规定》中规定ab什么是 电力调度数`据网络？"
remove_punctuation(line_test)

'电力监控系统安全防护123规定中规定ab什么是电力调度数据网络'

In [31]:
#读取停用词，形成停用词表，类型为list
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

In [32]:
#读取文本数据和停用词文件,采用正则化处理去除文本符号，并去停用词
def read_file(filenane):
    #######***************************答题区2***************************#######
    ###-----------------------------------10分
    file_txt = pd.read_excel(filenane, header=0)  
    file_txt = file_txt.dropna()  
    #停用词加载
    stopwords = stopwordslist("stopwords.txt")
    #正则化处理去除文本符号
    file_txt['clean_review'] = file_txt['题干'].apply(remove_punctuation)
    #######************************************************************#######
    return file_txt,stopwords
xls = '电力调度问答.xlsx'
file_txt,stopwords = read_file(xls)

In [33]:
#处理表格中的数据
def build_corpus(file_txt):
    file_txt['cut_review'] = file_txt['clean_review'].apply(lambda x: [w for w in list(jieba.cut(x)) if w not in stopwords and len(w) > 1])
    return file_txt
corpus = build_corpus(file_txt)
print(("file_txt['cut_review']:\n",corpus['cut_review'].values[0:5]))
#print(file_txt)

("file_txt['cut_review']:\n", array([list(['简述', '省级', '电网', '调度', '数据网', '组成部分', '功能', '部分', '之间', '互联', '路由器', '层面', '保证', '路由', '优化', '符合', '访问', '规则']),
       list(['简述', '技术', '层面', '防止', '非法', '远程', '登录', '电力', '调度', '数据网', '设备']),
       list(['电力', '监控', '系统安全', '防护', '规定', '规定', '电力', '调度', '数据网络']),
       list(['IP', '协议', '配置', '基本', '原则']),
       list(['路由器', '三种', '不同', '方式', '获得', '到达', '目的', '端的', '路径'])],
      dtype=object))


## Part2 构建向量表示

In [40]:
#构建TFIDF向量表示和哦OneHot向量表示
def build_tfidf(corpus):
    words_vocab =[]#存放不重复词语的列表
    #建立R行C列的矩阵TfIdf和OneHot存放tfidf向量表示和01向量表示，R为文档样本数，这里是行数，C为不重复词语数，即编码维度
    #######***************************答题区3***************************#######
    ###-----------------------------------10分

    for sentence in corpus['cut_review']:
        words_vocab.extend(sentence)
    words_vocab = sorted(set(words_vocab),key=words_vocab.index)#去掉重复的单词，构造词表
    
    ###-----------------------------------10分
    R=len(corpus)
    C=len(words_vocab)
    OneHot=np.zeros((R,C)) # 初始化全零二维矩阵
    Tf=np.zeros((R,C))

    for idx, sentence in enumerate(corpus['cut_review']):
        for word in sentence:
            if word in words_vocab:
                pos = words_vocab.index(word)
                OneHot[idx][pos]=1
                Tf[idx][pos]+=1# tf,统计某词语在一条样本中出现的次数
    ###-----------------------------------10分
    #计算tf-idf值
    #tf值
    row_sum=Tf.sum(axis=1) # 行相加，得到每个样本出现的词语数
    # 计算TF(t,d)
    tf=Tf/row_sum[:,np.newaxis] #分母表示各样本出现的词语数，tf为单词在样本中出现的次数，[:,np.newaxis]作用类似于行列转置
    # 计算DF(t,D)，IDF
    Df=OneHot.sum(axis=0) # 列相加，表示有多少样本包含词袋某词
    Idf=list(map(lambda x:math.log10((R+1)/(x+1)),Df))
    # 计算TFIDF
    TfIdf=Tf*np.array(Idf)
    #######************************************************************#######
    TfIdf=pd.DataFrame(TfIdf,columns=words_vocab)
    OneHot = pd.DataFrame(OneHot,columns=words_vocab)
    return TfIdf,OneHot,words_vocab

TfIdf,OneHot,words_vocab = build_tfidf(corpus)
   

In [11]:
def words_vector(query, Vector, words_vocab):
    vector_query = np.zeros(len(words_vocab))
    #给定用户问题的单词列表query，通过向量表示Vector和词表words_vocab生成用户问题的向量表示vector_query
    #######***************************答题区4***************************#######
    ###-----------------------------------10分
    Df=Vector.sum(axis=0) # 列相加，表示有多少样本包含词袋某词
    R = len(Vector)
    Idf=list(map(lambda x:math.log10((R+1)/(x+1)),Df))#words_vocab中所有单词的idf值
    for word in set(query):
        if word in words_vocab:
            vector_query[words_vocab.index(word)]= query.count(word)*Idf[words_vocab.index(word)]
    #######************************************************************#######
    return vector_query
                                                                                

## Part3 文本距离计算

In [15]:
#句子间距离计算方法
def sentences_distance(vec_1, vec_2):
    """
    :return: 两句文本的相识度
    """
    #余弦公式 Cosin Distance
    #######***************************答题区5***************************#######
    ###-----------------------------------5分
    num = vec_1.dot(vec_2.T)
    denom = np.linalg.norm(vec_1) * np.linalg.norm(vec_2)
    cos = num / denom
    #######************************************************************#######
    #欧式距离Euclidean Distance
    #######***************************答题区5***************************#######
    ###-----------------------------------5分
    #Euc=np.sqrt(np.sum(np.square(vector1-vector2)))
    euc=np.linalg.norm(vec_1-vec_2)
    #######************************************************************#######
    #曼哈顿距离Manhattan Distance
    #######***************************答题区5***************************#######
    ###-----------------------------------5分
    #Man=np.sum(np.abs(vector1-vector2))
    man=np.linalg.norm(vec_1-vec_2,ord=1)
    #######************************************************************#######

    return cos,euc,man

## Part4 问答系统（用户查询与候选问题）

In [17]:
#所有问题组合起来的倒排表 result,格式如下
#result={word1:[问题1,问题2,...],word2:[问题1,问题3,...]...}
#即记录每个词分别出现在第几个问题中
result = {}
for i in range(len(corpus)):
    #print(corpus.iloc[i]['cut_review'])
    idx, words = i, corpus.iloc[i]['cut_review']
    for word in words:#words in each candicate questions
        if word in result.keys():
            result[word].append(idx)
        else:
            result[word]= [idx]
#print(result)

In [18]:
#通过用户问题从问题集合中选出候选问题
sentence='电力监控系统中的调度数据网络是什么'
clean_reviewyonghu = remove_punctuation(sentence)  # 用户问题去除标点
cut_reviewyonghu = [w for w in list(jieba.cut(clean_reviewyonghu)) if
                        w not in stopwords and len(w) > 1]  # 用户问题去除停用词，单字词 得到关键词
print(cut_reviewyonghu)
#查找用户问题关键词在数据库中对应的问题id
Problem_Id = []
for word in cut_reviewyonghu:
    if word in result.keys():
        Problem_Id.extend(result[word])
id = (list(set(Problem_Id)))  # 去重之后的ID

['电力', '监控', '系统', '调度', '数据网络']


In [42]:
#计算余弦相似度
query = sentence  # 用户所提问题
similarity_cos = {}  # 存储余弦相似度结果
similarity_euc = {}  # 存储欧式距离结果
similarity_man = {}  # 存储曼哈顿距离结果
if len(id) == 0:
    print('数据库里没有该问题，请重新提问')
else:
    #使用函数cosine_similarity求出与用户问题最相似的问题，
    #排序输出和用户问题相似度最高的前3个问题的答案案
    #这里我们采用TfIdf的向量表示
#######***************************答题区6***************************#######
###-----------------------------------15分
    for index in id:
        candicate = corpus.iloc[index]['题干']
        vec_1 = words_vector(cut_reviewyonghu,OneHot,words_vocab)
        vec_2 = TfIdf.loc[index].values[0:]
        simil_cos,simil_euc,simil_man = sentences_distance(vec_1, vec_2)  # 余弦相识度
        #print('用户查询和问题{0}的相似度是：{1}'.format(candicate, simil_value))
        similarity_cos[index] = simil_cos
        similarity_euc[index] = simil_euc
        similarity_man[index] = simil_man
#输出和用户问题相似度最高几个问题的答案
###-----------------------------------10分
res_cos = sorted(similarity_cos.items(), key=lambda d: d[1], reverse=True)[:3]  # 降序
res_euc = sorted(similarity_euc.items(), key=lambda d: d[1], reverse=False)[:3]  # 升序
res_man = sorted(similarity_man.items(), key=lambda d: d[1], reverse=False)[:3]  # 升序
#######************************************************************#######
#res_cos,res_euc,res_man分别存放的是排序后的相似问题及相似度分数

#打印输出结果
print('用户所提的问题是：', query,'\n')
print('********************************************************************')
#余弦相似度排名
print("和用户问题相似度最高的前3个问题:\n",res_cos)
for i, j in res_cos:
    print('数据库相似的问题是:{0} {1} \n 答案是:{2} \n 相似度是:{3}\n'.format(i,corpus.iloc[i]['题干'], corpus.iloc[i]['答案'],j))
print('********************************************************************')
#欧式距离排名
print("和用户问题相似度最高的前3个问题:\n",res_euc)
for i, j in res_euc:
     print('数据库相似的问题是:{0} {1} \n 答案是:{2} \n 相似度是:{3}\n'.format(i,corpus.iloc[i]['题干'], corpus.iloc[i]['答案'],j))
print('********************************************************************')
#曼哈顿距离排名
print("和用户问题相似度最高的前3个问题:\n",res_man)
for i, j in res_man:
     print('数据库相似的问题是:{0} {1} \n 答案是:{2} \n 相似度是:{3}\n'.format(i,corpus.iloc[i]['题干'], corpus.iloc[i]['答案'],j))
print('********************************************************************')

用户所提的问题是： 电力监控系统中的调度数据网络是什么 

********************************************************************
和用户问题相似度最高的前3个问题:
 [(2, 0.6513533036883628), (11, 0.2933322579396123), (14, 0.27594449056341647)]
数据库相似的问题是:2 《电力监控系统安全防护规定》中规定什么是电力调度数据网络？ 
 答案是:电力调度数据网络，是指各级电力调度专用广域数据网络、电力生产专用拨号网络等（14号令如此规定） 
 相似度是:0.6513533036883628

数据库相似的问题是:11 根据国网（调/4）335-2014 《 国家电网公司电力调度自动化系统运行管理规定》，自动化系统出现在哪些情况下应立即向上级电力调度机构汇报？  
 答案是:（1）由于自动化系统原因导致电网发生《国家电网公司安全事故调查规程》中规定的5-7级事件；
（2）调度技术支持系统全停；
（3）省调及以上调度技术支持系统主要功能失效超过1小时，地、县调调度技术支持系统主要功能失效超过4小时；
（4）子站设备主要功能连续故障停止运行超过24小时，故障停止运行时间指从对其有调度管辖权的调度机构自动化值班人员发出故障通知时算起，到故障消除、恢复使用时止；
（5）数据通信中断厂站数量超过10个且中断时间超过半小时；
（6）影响调度自动化系统正常运行的电力二次系统安全事件。 
 相似度是:0.2933322579396123

数据库相似的问题是:14 《国家电网公司电力调度自动化系统运行管理规定》中对于调度自动化系统缺陷的处理时限有何要求？ 
 答案是:缺陷处理时间要求：紧急缺陷4小时内处理；重要缺陷24小时内处理；一般缺陷2周内消除。 
 相似度是:0.27594449056341647

********************************************************************
和用户问题相似度最高的前3个问题:
 [(93, 3.8054992930016587), (92, 3.903840334191012), (2, 4.105729415750348)]
数据库相似的问题